In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2,RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

In [2]:
df = pd.read_csv("train.csv", header=None)
new_cols = ['C' + str(i) for i in range(1, 14 + 1)] + ['X' + str(i) for i in range(1, 19 + 2)]
new_cols[-1] = 'Y'
df.columns = new_cols

features = df.loc[:, df.columns != 'Y']
numerical_features = df.select_dtypes(include=['float64']).columns
categorical_features = df.select_dtypes(include=['object']).columns
target = df['Y']

df

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,X11,X12,X13,X14,X15,X16,X17,X18,X19,Y
0,V1,V1,V1,V1,V1,V1,V1,V1,V1,V1,...,-0.619,-0.536,-0.092,0.182,0.034,-0.172,0.401,0.393,0.216,1
1,V1,V1,V1,V1,V1,V1,V2,V1,V3,V5,...,0.352,0.073,-0.092,1.098,0.034,1.160,0.401,0.037,0.216,1
2,V1,V1,V1,V1,V1,V1,V1,V1,V2,V2,...,-1.933,-0.536,-0.092,1.098,0.034,0.716,0.401,0.724,0.216,-1
3,V1,V1,V1,V1,V1,V1,V1,V1,V2,V2,...,-0.762,-0.536,-0.092,1.098,0.034,0.716,0.401,0.712,0.216,-1
4,V1,V1,V1,V1,V1,V1,V1,V1,V1,V1,...,-0.505,-0.536,-0.092,0.182,0.034,0.716,0.401,0.393,0.216,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,NaN,NaN,NaN,V3,V1,NaN,NaN,NaN,V1,V4,...,NaN,NaN,NaN,-2.568,0.034,NaN,0.401,1.890,0.216,1
396,NaN,NaN,NaN,V3,V2,NaN,NaN,V1,V3,V2,...,NaN,NaN,NaN,-4.401,-2.684,NaN,0.401,1.391,0.216,1
397,NaN,NaN,NaN,V3,V2,NaN,NaN,NaN,V1,V4,...,NaN,NaN,NaN,0.548,4.926,NaN,0.401,0.992,0.216,1
398,NaN,NaN,NaN,V3,V1,NaN,NaN,NaN,V1,V8,...,NaN,NaN,NaN,-1.651,3.567,NaN,0.401,0.992,0.216,1


In [3]:
print((np.count_nonzero(target == 1)/np.count_nonzero(target == -1)) * 100)

68.77637130801688


In [4]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2)

In [5]:
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

#TODO Experiment with strategies
iterative_numerical_imputer = IterativeImputer(max_iter=10)
iterative_numerical_imputer.fit(X_train[numerical_features])
X_train.loc[:, numerical_features] = iterative_numerical_imputer.transform(X_train.loc[:, numerical_features])
X_test.loc[:, numerical_features] = iterative_numerical_imputer.transform(X_test.loc[:, numerical_features])

categorical_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
categorical_imputer.fit(X_train[categorical_features])
X_train.loc[:, categorical_features] = categorical_imputer.transform(X_train.loc[:, categorical_features])
X_test.loc[:, categorical_features] = categorical_imputer.transform(X_test.loc[:, categorical_features])

C:\Users\Gábor\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\impute\_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"
C:\Users\Gábor\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
C:\Users\Gábor\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [6]:
X_train_len = len(X_train)
X = pd.concat([X_train, X_test])
X = pd.get_dummies(X)
X_train = X[:X_train_len]
X_test = X[X_train_len:]

In [7]:
#X.to_csv("save\X.csv", index=False)
#X_train.to_csv("save\X_train.csv", index=False)
#X_test.to_csv("save\X_test.csv", index=False)

In [8]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier

classifier1 = RandomForestClassifier(n_estimators=1000)
classifier1.fit(X_train, y_train)
print("Test Score: ", classifier1.score(X_test, y_test))

classifier2 = ExtraTreesClassifier(n_estimators=1000)
classifier2.fit(X_train, y_train)
print("Test Score: ", classifier2.score(X_test, y_test))

Test Score:  0.8
Test Score:  0.825


In [9]:
y_pred1 = classifier1.predict(X_test)
cm1 = confusion_matrix(y_test, y_pred1)

print("RandomForestClassifier")
print(cm1)
print(classification_report(y_test, y_pred1))


y_pred2 = classifier2.predict(X_test)
cm2 = confusion_matrix(y_test, y_pred2)

print("ExtraTreesClassifier")
print(cm2)
print(classification_report(y_test, y_pred2))

RandomForestClassifier
[[48  3]
 [13 16]]
              precision    recall  f1-score   support

          -1       0.79      0.94      0.86        51
           1       0.84      0.55      0.67        29

    accuracy                           0.80        80
   macro avg       0.81      0.75      0.76        80
weighted avg       0.81      0.80      0.79        80

ExtraTreesClassifier
[[47  4]
 [10 19]]
              precision    recall  f1-score   support

          -1       0.82      0.92      0.87        51
           1       0.83      0.66      0.73        29

    accuracy                           0.82        80
   macro avg       0.83      0.79      0.80        80
weighted avg       0.83      0.82      0.82        80



In [10]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred1))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred1))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred1)))

Mean Absolute Error: 0.4
Mean Squared Error: 0.8
Root Mean Squared Error: 0.8944271909999159


In [11]:
def evaluate_model(X_train, y_train, X_test, y_test, model):
	cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
	#model.fit(X_train, y_train)
	return cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

models, names = list(), list()
# SVM
models.append(SVC(gamma='auto'))
names.append('SVM')
# KNN
models.append(KNeighborsClassifier())
names.append('KNN')
# Neutral Network
#models.append(MLPClassifier(solver = 'adam', max_iter=1000, tol=0.000001, early_stopping=True, validation_fraction=0.1, n_iter_no_change=20))
#names.append('NEU')
# Bagging
models.append(BaggingClassifier(n_estimators=1000))
names.append('BAG')
# RF
models.append(RandomForestClassifier(n_estimators=1000))
names.append(' RF')
# ET
models.append(ExtraTreesClassifier(n_estimators=1000))
names.append(' ET')


results = list()
for i in range(len(models)):
	scores = evaluate_model(X_train, y_train, X_test, y_test, models[i])
	results.append(scores)
	print('>%s %.3f (%.3f)' % (names[i], scores.mean(), scores.std()))
	

>SVM 0.746 (0.047)
>KNN 0.765 (0.050)
>BAG 0.801 (0.034)
> RF 0.818 (0.028)
> ET 0.830 (0.040)


In [12]:
from sklearn.model_selection import RandomizedSearchCV

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Create the random grid
random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 1000, stop = 2000, num = 5)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': max_depth,
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

print(random_grid)

{'n_estimators': [1000, 1250, 1500, 1750, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [13]:
import time

rf = RandomForestClassifier()

# Random search of parameters, using 3 fold cross validation, search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
start = time.time()

rf_random.fit(X_train, y_train)

end = time.time()
print("Elapsed time: ", end - start, " seconds, which is: ", (end - start) / 60, " minutes")

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Elapsed time:  141.53899812698364  seconds, which is:  2.358983302116394  minutes


In [14]:
rf_random.best_params_

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 90,
 'bootstrap': False}

In [15]:
"""
def evaluate(model, X_test, y_test):
    #TODO swap to cross_val
    y_pred = model.predict(X_test)
    errors = abs(y_pred - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    print('\nModel Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    print(confusion_matrix(y_test, y_pred))
    #print(classification_report(y_test, y_pred))
    
    return accuracy
"""
def evaluate(model, X_test, y_test):
    score = model.score(X_test, y_test)
    print('Accuracy = {:0.2f}%.'.format(score))
    return score
    
base_model = RandomForestClassifier(n_estimators = 100)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

random_accuracy = evaluate(rf_random.best_estimator_, X_test, y_test)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Accuracy = 0.82%.
Accuracy = 0.81%.
Improvement of -1.52%.


In [16]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True, False],
    'max_depth': [40, 50, 60],
    'max_features': ['auto'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [4, 5, 6],
    'n_estimators': [1250, 1500, 2000]
}

# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

In [17]:
start = time.time()

grid_search.fit(X_train, y_train)

end = time.time()
print("Elapsed time: ", end - start, " seconds, which is: ", (end - start) / 60, " minutes")

grid_search.best_params_

Fitting 3 folds for each of 162 candidates, totalling 486 fits
Elapsed time:  277.8500006198883  seconds, which is:  4.630833343664805  minutes


{'bootstrap': True,
 'max_depth': 50,
 'max_features': 'auto',
 'min_samples_leaf': 3,
 'min_samples_split': 6,
 'n_estimators': 2000}

In [18]:
# Fit the grid search to the data

grid_accuracy = evaluate(grid_search.best_estimator_, X_test, y_test)

print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

Accuracy = 0.79%.
Improvement of -4.55%.


In [19]:
# TODO Cross valudate

In [21]:
#Predicting the original Test Set
test_to_predict = pd.read_csv("test.csv", header=None)

numerical_features = test_to_predict.select_dtypes(include=['float64']).columns
categorical_features = test_to_predict.select_dtypes(include=['object']).columns

In [22]:
test_to_predict.loc[:, numerical_features] = iterative_numerical_imputer.transform(test_to_predict.loc[:, numerical_features])
test_to_predict.loc[:, categorical_features] = categorical_imputer.transform(test_to_predict.loc[:, categorical_features])

test_to_predict_len = len(test_to_predict)
original = pd.concat([features, test_to_predict])
original = pd.get_dummies(original)
test_to_predict = X[:test_to_predict_len]

In [23]:
test_to_predict

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,C12_V1,C12_V2,C12_V3,C13_V1,C13_V2,C13_V3,C13_V4,C13_V5,C14_V1,C14_V2
13,-0.557000,-1.235,-1.092000,0.344,-0.845,1.425,-0.099,-0.844000,0.537,1.234000,...,0,0,1,0,1,0,0,0,1,0
63,-0.280000,-1.235,-1.092000,-1.064,0.189,-0.513,-1.196,0.237000,-0.223,-0.438000,...,1,0,0,1,0,0,0,0,1,0
394,-0.013959,0.627,0.348000,2.301,0.189,1.425,0.120,0.110888,0.233,-0.137258,...,1,0,0,1,0,0,0,0,0,1
229,1.102000,-0.366,-0.617000,2.066,-0.328,-2.451,-0.757,0.616000,-0.680,0.784000,...,1,0,0,0,0,0,0,1,0,1
348,-0.557000,-1.359,-0.203325,0.266,0.835,-2.128,-0.428,-0.216000,0.689,-0.438000,...,1,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,1.378000,0.006,-0.127000,0.188,0.318,0.133,-0.648,0.587000,0.081,-1.194000,...,1,0,0,1,0,0,0,0,1,0
367,0.179922,1.123,0.348000,-0.282,0.447,-1.159,-0.976,-0.021306,-0.071,-0.051728,...,0,0,1,1,0,0,0,0,1,0
260,-0.004000,-1.111,-0.127000,-0.360,-0.458,1.425,-0.648,0.689000,-0.497,-0.546000,...,0,0,1,0,1,0,0,0,0,1
213,1.102000,-0.118,-2.056000,-1.064,0.189,-0.513,0.120,0.324000,0.537,0.784000,...,1,0,0,1,0,0,0,0,0,1


In [48]:
prediction = grid_search.best_estimator_.predict(test_to_predict)
print("Count(1): ", np.count_nonzero(prediction == 1), "Count(-1): ", np.count_nonzero(prediction == -1))
print((np.count_nonzero(prediction == 1)/np.count_nonzero(prediction == -1)) * 100)

Count(1):  40 Count(-1):  60
66.66666666666666


In [49]:
# Exporting
prediction = pd.DataFrame(prediction)
prediction.to_csv("predictions.csv", index=False, header=False)

In [26]:
# TODO https://datascience.stackexchange.com/questions/75345/need-help-understanding-data-leakage
# TODO https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74